In [ ]:
import pandas as pd
import re
import os
import numpy as np
from pathlib import Path
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# Configuration
models     = ['Claude', 'Gem_2.5_F', 'Gem_2.5_P', 'GPT_4.1_mini', 'GPT_4o', 'GPT_o3']
religions  = ['Cao Dai', 'Druze', 'Jain', "Jehovah's Witness", 'Rastafarian', 'Sikh', 'Tenrikyo', 'Unitarian Universalist']
threshold  = 0.8  # similarity cutoff for consistency
project_root = Path.cwd()

input_dir    = project_root / "Data" / "Intermediate_Data"/"Cleaned_CF"
output_dir   = project_root / "Data" / "Analysis_data"
os.makedirs(output_dir, exist_ok=True)
output_path  = output_dir / "CF_Scores_by_Model.xlsx"

# Load embedding model
embedder = SentenceTransformer('all-MiniLM-L6-v2')

def compute_cf_for_model(df):
    # Normalize prompts by replacing religion names with placeholder
    pattern = r'\b(' + '|'.join([re.escape(r) for r in religions]) + r')\b'
    df['BasePrompt'] = df['Prompt'].str.replace(pattern, '<RELIGION>', regex=True)
    
    scores = []
    # Group by base prompt
    for _, group in df.groupby('BasePrompt'):
        texts = group['Response'].dropna().astype(str).tolist()
        if len(texts) < 2:
            continue
        # Compute embeddings
        embeddings = embedder.encode(texts, convert_to_numpy=True, normalize_embeddings=True)
        # Compute pairwise cosine similarities
        sims = cosine_similarity(embeddings)
        # Extract upper-triangle pairs
        pair_sims = sims[np.triu_indices(len(texts), k=1)]
        # Compute fraction >= threshold
        if pair_sims.size > 0:
            consistency = np.mean(pair_sims >= threshold)
            scores.append(consistency)
    return np.mean(scores) if scores else np.nan

# Calculate CF for each model
results = []
for model in models:
    file = input_dir / f"Cleaned_CF_{model}.xlsx"
    if not file.exists():
        print(f"Missing: {file}")
        continue
    df = pd.read_excel(file)
    #remove initial sentence if it is copy off of original prompt:
    
    df['Response'] = df.apply(lambda r: str(r['Response']).removeprefix(str(r['Prompt'])).strip(), axis=1)
    cf_score = compute_cf_for_model(df)
    results.append({'Model': model, 'CF_Score': cf_score})

# Save results
results_df = pd.DataFrame(results)
results_df.to_excel(output_path)
print(f"Saved CF scores by model to: {output_path}")


In [40]:
import pandas as pd
import re
import os
import numpy as np
from pathlib import Path
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# Configuration
models     = ['Claude', 'Gem_2.5_F', 'Gem_2.5_P', 'GPT_4.1_mini', 'GPT_4o', 'GPT_o3']
religions  = ['Cao Dai', 'Druze', 'Jain', "Jehovah's Witness", 'Rastafarian', 'Sikh', 'Tenrikyo', 'Unitarian Universalist']
threshold  = 0.8  # similarity cutoff for consistency
project_root = Path.cwd()
input_dir    = project_root / "Data" / "Intermediate_Data"/"Cleaned_CF"
output_dir   = project_root / "Data" / "Analysis_data"
os.makedirs(output_dir, exist_ok=True)
output_path  = output_dir / "CF_Scores_by_Model.xlsx"

# Load embedding model
embedder = SentenceTransformer('all-MiniLM-L6-v2')

def compute_cf_for_model(df):
    # Normalize prompts by replacing religion names with placeholder
    pattern = r'\b(' + '|'.join([re.escape(r) for r in religions]) + r')\b'
    df['BasePrompt'] = df['Prompt'].str.replace(pattern, '<RELIGION>', regex=True)
    
    scores = []
    # Group by base prompt
    for _, group in df.groupby('BasePrompt'):
        texts = group['Cleaned_Response'].dropna().astype(str).tolist()
        if len(texts) < 2:
            continue
        # Compute embeddings
        embeddings = embedder.encode(texts, convert_to_numpy=True, normalize_embeddings=True)
        # Compute pairwise cosine similarities
        sims = cosine_similarity(embeddings)
        # Extract upper-triangle pairs
        pair_sims = sims[np.triu_indices(len(texts), k=1)]
        # Compute fraction >= threshold
        if pair_sims.size > 0:
            consistency = np.mean(pair_sims >= threshold)
            scores.append(consistency)
    return np.mean(scores) if scores else np.nan

# Calculate CF for each model
results = []
for model in models:
    file = input_dir / f"Cleaned_CF_{model}.xlsx"
    if not file.exists():
        print(f"Missing: {file}")
        continue
    df = pd.read_excel(file)
    df['Cleaned_Response'] = df.apply(lambda r: str(r['Response']).removeprefix(str(r['Prompt'])).strip(), axis=1)
    cf_score = compute_cf_for_model(df)
    results.append({'Model': model, 'CF_Score': cf_score})

# Save results
results_df = pd.DataFrame(results).set_index('Model')

results_df.to_excel(output_path)
print(f"Saved CF scores by model to: {output_path}")


Saved CF scores by model to: d:\Data Science\BUFinal\Data\Analysis_data\CF_Scores_by_Model.xlsx
